In [ ]:
%pip install dspy-ai[qdrant]
!pip install ollama

In [ ]:
!pip install --upgrade protobuf google-cloud-aiplatform

In [ ]:
!pip install -U qdrant-client phoenix

In [1]:
from qdrant_client import QdrantClient

# Option 1: Connecting to a Remote Qdrant Server

# Replace "URL" with the actual URL of your remote Qdrant server
# Replace "API_KEY" with your Qdrant API key
# qdrant_client = QdrantClient(
#     url="URL",
#     api_key="API_KEY",
# )

# Option 2: Connecting to an In-Memory Qdrant Instance (for development)

# This option launches a temporary Qdrant instance in memory, useful for development purposes.
# qdrant_client = QdrantClient(":memory:")

# Option 3: Connecting to a Local Qdrant Server

# This option connects to a Qdrant server running on the same machine (localhost) on port 6333 (default port).
qdrant_client = QdrantClient(host='localhost', port=6333)

print(qdrant_client.get_collections())

c:\Users\ashis\projects\Blog\Qdrant-DsPy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


collections=[CollectionDescription(name='resume-collection')]


In [2]:
from datasets import load_dataset

# Load the dataset from HuggingFace
dataset = load_dataset("DevashishBhake/resume_section_classification")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['ID', 'Category', 'Resume'],
        num_rows: 1219
    })
})


In [3]:
import random

rows = dataset["train"]
rows_list = list(rows)

random_rows = random.sample(rows_list, 10)  
resume_content = []

# Iterate over the random rows and append the resume content to the list
for row in random_rows:
    resume_content.append(row["Resume"])

resume_content    

["b'Sumit Khemka\\n\\nBangalore, India\\nkhemkasumit@gmail.com\\n+91 9886632360\\n\\nPROFESSIONAL SUMMARY\\n\\n\\xef\\x82\\xb7\\n\\n8.5+ years of experience in Information Technology, with 2 years at London (UK) and Houston (US).\\n\\n\\xef\\x82\\xb7\\n\\nDomain knowledge in Data Quality, Market Risk and Pricing (MVAR), Health Quality Measures (MU Quality\\nReporting), Mortgage Servicing.\\n\\n\\xef\\x82\\xb7\\n\\nExperience in Shared Services, Matrix organization and Agile model of project execution.\\n\\n\\xef\\x82\\xb7\\n\\nCertified MCP and MCTS in SQL Server and Business Intelligence and .NET 2.0.\\n\\n\\xef\\x82\\xb7\\n\\n7 years of experience in Data warehousing and Business Intelligence tools.\\n\\n\\xef\\x82\\xb7\\n\\n6 years strong experience with SQL Server 2005/2008/2012 databases.\\n\\n\\xef\\x82\\xb7\\n\\n6 years strong experience with Microsoft Business Intelligence Suite .ie SSIS,SSRS & SSAS.\\n\\n\\xef\\x82\\xb7\\n\\n2 years working experience on Oracle Databases.\\n\\

In [5]:
ids = list(range(0, 10))
documents = resume_content
qdrant_client.add(
    collection_name="resume-collection",
    documents=documents,
    ids=ids,
)
print(qdrant_client.get_collections())

collections=[CollectionDescription(name='resume-collection')]


In [24]:
from dspy.retrieve.qdrant_rm import QdrantRM
from qdrant_client import QdrantClient, models
qdrant_retriever = QdrantRM(
    qdrant_collection_name="resume-collection",
    qdrant_client=qdrant_client,
)

In [25]:
# !ollama run llama3:8b
results = qdrant_retriever("Candidate whow worked in google", k=3)

for result in results:
    print("Document:", result.long_text, "\n")

Document: b'Barbara Ann Jones\n(516) 555-5555 \xef\x82\x98 someone@somedomain.com \xef\x82\x98 LinkedIn \xef\x82\x98 Facebook\nTwitter Handle \xef\x82\x98 My del.icio.us \xef\x82\x98 My StumbleUpon \xef\x82\x98 Podcast Links\nSkype: SomeSkypeName \xef\x82\x98 Link to download text and PDF resumes\n\n\xef\x82\x98\n\nBlog\n\nSOCIAL MEDIA MANAGER\n\nwww.myportfolio.com\n\nChampion of social media tools and technologies, with a track record of creating and implementing\nsuccessful social media programs. Keep up-to-date with constantly evolving technologies in online\nsocial networking, the blogosphere, search tools and Web 2.0, and work closely with clients to\ncreate innovative, effective campaigns.\n\nPartial List of Tools (full list available at myportfolio.com)\n\xef\x82\x97\n\xef\x82\x97\n\xef\x82\x97\n\xef\x82\x97\n\xef\x82\x97\n\xef\x82\x97\n\xef\x82\x97\n\xef\x82\x97\n\nBlogger\nWordPress\nTypePad\nSix Apart\nLive Writer\nPodcasting/\nAudacity\nFeedBlitz\nBloglines\n\n\xef\x82\x97\

In [26]:
import dspy
from groq import Groq
# ollama_model = dspy.OllamaLocal(model="llama3:8b",model_type='text',
#                                 max_tokens=350,
#                                 temperature=0.1,
#                                 top_p=0.8, frequency_penalty=1.17, top_k=40, timeout=1000)
llm = dspy.GROQ(model='llama3-8b-8192', api_key ="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" )

In [27]:
llm("Say Hi!!")

['HI!!!']

In [28]:
dspy.configure(lm=llm, rm=qdrant_retriever)

## Structured Data Extraction

In [32]:
class Candidate(dspy.Signature):
    """
    You are a resume parser. Parse this textual resume and return the extracted information as a single JSON string with the following format:

    {
        "name": "<candidate name>",
        "email": "<candidate email>",
        "years_of_experience": <number of years>
    }
    """
    description = dspy.InputField(desc="Textual content of the candidate's resume. This should be plain text containing the candidate's work experience, skills, and other relevant information.")
    parsed_content= dspy.OutputField(desc="Extracted JSON string. This must follow a json format with three fields name, email and years_of_experience only!")



In [33]:
c = Candidate(description="b'BRAD MISTRY\nGRAPHIC DESIGNER\n\nbradmistry@gmail.com\nwww.bradmistry.blogspot.ca\nwww.linkedin.com/in/bradmistry\n\n647.967.4580\nVancouver, BC\n\nOBJECTIVES\nLooking for a position as a Graphic Designer, working with project leads to execulte content in respect to the design criteria.\n\nEDUCATION\nGAME DESIGN George Brown College\n2013\nCertificate in Game Design.\nCourse work related to designing games and game\nculture. Designed two playable mobile games.\n\nINDUSTRIAL DESIGN OCAD University\n2008\nBachelor of Design.\nWhere I discovered I have a natural talent for Graphic Design.\nStrategic Brand Deveopment was my favourite course.\n\nSKILLS\nGRAPHIC DESIGN\nBRAND STRATEGY\nART DIRECTION\nINTERFACE DESIGN\nGAME DESIGN\nINDUSTRIAL DESIGN\nADOBE\nMAC\n\nOver five years of freelance work for a diversity of corporate and personal clients.\nConcepted and designed several brand campaigns consisting of logos, stationary, magazine ads, etc.\nManaged a development pool of over fifty artists to execute the look and feel of two mobile games.\nMapped out and developed the interface for a two mobile games.\nDeveloped two playable mobile games. Experience entailed art direction, client meetings, documentation.\nConcepted several products for practicing professionals in the culinary and make-up industry.\nExpert knowledge of Adobe Illustrator and Photoshop.\nFluid with OSX and equipped with the latest licensed version of Adobe Master Suite and Microsoft Office.\n\nWORK HISTORY Corporate\n\nToolKit\n\n2012\n\nGRAPHIC DESIGNER\nGeorge Brown College\nDeveloped partnership presentations\nand documentation to match branding\nstandards.\nClient\nGeorge Brown Admin\n\nPediatric Resident Guide\n\n2009\n\nPaid Summer Intern\n\n2006\n\nGRAPHIC DESIGNER\nUniversity of Toronto: Faculty of Medicine\nDeveloped a nine page printed hand\nguide that outlines standard pediatric\nprocedures.\nClient\nToronto SickKids Hospital\n\nJUNIOR DESIGNER\nGottschalk + Ash Int\xc3\xa2\xc2\x80\xc2\x99l\nWorked in this branding and wayfinding\nstudio assisting the design director for\nconceptual work.\nClients\nCanadian Museum of Nature\nOntario Lottery and Gaming\nRyerson University\n\nCheftotes\n\nMakeup-To-Go\n\nFreelance\nTimescape\n\n2013\n\nConcept and Graphic Design\nConcepted and developed a storyboard\npresentation for a web app idea.\n\n2008\n\nIndustrial Design\nDesigned and developed a carrying case\nfor kitchen utensils used by chefs.\n\n2007\n\nIndustrial Design\nDesigned and developed a product line\nof portable, refillable make-up.'", parsed_content="{\"name\": \"BRAD MISTRY\",\"email\": \"bradmistry@gmail.com\",\"yearsOfExperience\": \"5+\"}")
print(c.model_construct)

<bound method BaseModel.model_construct of Candidate(description -> parsed_content
    instructions='\n    You are a resume parser. Parse this textual resume and return the extracted information as a single JSON string with the following format:\n\n    {\n        "name": "<candidate name>",\n        "email": "<candidate email>",\n        "years_of_experience": <number of years>\n    }\n    '
    description = Field(annotation=str required=True json_schema_extra={'desc': "Textual content of the candidate's resume. This should be plain text containing the candidate's work experience, skills, and other relevant information.", '__dspy_field_type': 'input', 'prefix': 'Description:'})
    parsed_content = Field(annotation=str required=True json_schema_extra={'desc': 'Extracted JSON string. This must follow a json format with three fields name, email and years_of_experience only!', '__dspy_field_type': 'output', 'prefix': 'Parsed Content:'})
)>


In [34]:
class CandidateExtractor(dspy.Module):

    def __init__(self):
        super().__init__()
        # Retrieve module to get relevant documents
        self.retriever = qdrant_retriever
        # Predict module for the created signature
        self.predict = dspy.Predict(Candidate)

    def forward(self, query: str):
        # Retrieve the most relevant documents
        results = self.retriever(query)
        print(results)
        candidate = self.predict(description=results[0]["long_text"])
        return candidate.parsed_content

In [35]:
extractor = CandidateExtractor()
response = extractor("Get the details of candidate with business majors.")
response



[{'long_text': "b'Market Your Skills: Interpersonal,\\nAnalytical, Leadership, Communication\\n\\nSarah K. Davis\\n\\n1234 W. Michigan Ave \\xe2\\x80\\xa2 Chicago, IL 45670 \\xe2\\x80\\xa2 456-7890-1234 \\xe2\\x80\\xa2 sarahdavis@northwestern.edu\\n\\nEDUCATION\\nJune 2015\\nNorthwestern University, Evanston, IL\\nMaster of Science in Higher Education Administration and Policy\\nMaster\\xe2\\x80\\x99s Research Project: \\xe2\\x80\\x9cWhat on-the-job experiences at a college union relate most strongly with undergraduate\\nlearning outcomes?\\xe2\\x80\\x9d\\nJune 2012\\nHighlight any relevant experience within\\nUniversity of Virginia, Charlottesville, VA\\nhigher education through internships,\\nBachelor of Science in Psychology\\ngraduate assistantships, or volunteer work\\nMagna Cum Laude\\n\\nCAREER SERVICES EXPERIENCE\\nUniversity Career Services, Northwestern University\\nCareer Counseling Intern\\nAugust 2014 \\xe2\\x80\\x93 Present\\n\\xef\\x82\\xb7 Conduct internship advising to

'Here is the extracted information in the required JSON format:\n\n{\n    "name": "Sarah K. Davis",\n    "email": "sarahdavis@northwestern.edu",\n    "years_of_experience": 8\n}\n\nNote: I assumed the "years_of_experience" to be 8, as the resume mentions "June 2012" as the graduation date and "August 2014" as the start date of the current job, which is approximately 2 years ago.'

## TypedPredictors

In [36]:
from dspy.functional import TypedPredictor
import pydantic
class Selection(pydantic.BaseModel):
    name: str
    email_id: str
    years_of_experience: str

In [40]:
class Candidate(dspy.Signature):
    """
    You are a resume parser. Parse this textual resume and return the extracted information with the following fields only:
        "name": "<candidate name>",
        "email": "<candidate email>",
        "years_of_experience": <number of years>
    """
    description: str = dspy.InputField(desc="Textual content of the candidate's resume. This should be plain text containing the candidate's work experience, skills, and other relevant information.")
    parsed_content: Selection= dspy.OutputField(desc="Extracted information. This must have three fields: name, email and years_of_experience only!")



In [41]:
import functools
class CandidateExtractor(dspy.Module):

    def __init__(self):
        super().__init__()
        # Retrieve module to get relevant documents
        self.retriever = qdrant_retriever
        # Predict module for the created signature
        self.predict = dspy.functional.TypedPredictor(Candidate)

    def forward(self, query: str):
        # Retrieve the most relevant documents
        results = self.retriever(query)
        print(results)
        candidate = self.predict(description=results[0]["long_text"])
        return candidate.parsed_content

In [43]:
extractor = CandidateExtractor()
response = extractor("Get the details of candidate with business majors.")
response


[{'long_text': "b'Market Your Skills: Interpersonal,\\nAnalytical, Leadership, Communication\\n\\nSarah K. Davis\\n\\n1234 W. Michigan Ave \\xe2\\x80\\xa2 Chicago, IL 45670 \\xe2\\x80\\xa2 456-7890-1234 \\xe2\\x80\\xa2 sarahdavis@northwestern.edu\\n\\nEDUCATION\\nJune 2015\\nNorthwestern University, Evanston, IL\\nMaster of Science in Higher Education Administration and Policy\\nMaster\\xe2\\x80\\x99s Research Project: \\xe2\\x80\\x9cWhat on-the-job experiences at a college union relate most strongly with undergraduate\\nlearning outcomes?\\xe2\\x80\\x9d\\nJune 2012\\nHighlight any relevant experience within\\nUniversity of Virginia, Charlottesville, VA\\nhigher education through internships,\\nBachelor of Science in Psychology\\ngraduate assistantships, or volunteer work\\nMagna Cum Laude\\n\\nCAREER SERVICES EXPERIENCE\\nUniversity Career Services, Northwestern University\\nCareer Counseling Intern\\nAugust 2014 \\xe2\\x80\\x93 Present\\n\\xef\\x82\\xb7 Conduct internship advising to

ValueError: ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'parsed_content': "ValueError('json output should start and end with { and }')"})

In [ ]:
Selection(name='Sarah K. Davis', email_id='sarahdavis@northwestern.edu', years_of_experience=8)

## Assertions

In [44]:
import functools
import json

def assert_valid_json(candidate_json):
  """
  This function checks if the provided JSON string matches the expected format exactly.

  Expected format: {
      "name": "<candidate name>",
      "email": "<candidate email>",
      "years_of_experience": <number of years>
  }

  Args:
      candidate_json (str): The JSON string to be validated.

  Returns:
      bool: True if the JSON matches the format exactly, False otherwise.
  """

  try:
    # Parse the JSON string
    json_data = json.loads(candidate_json)

    # Check for exact key match and data types (optional)
    return (
        len(json_data) == 3 and  # Exactly 3 keys
        all(key in json_data for key in ["name", "email", "years_of_experience"]) and
        isinstance(json_data["name"], str) and  # Name as string (optional)
        isinstance(json_data["email"], str)  # Email as string (optional)
    )

  except json.JSONDecodeError:
    return False

    
failed_assertion_message = """
Output must be a single JSON string only, in this format!
{
        "name": "<candidate name>",
        "email": "<candidate email>",
        "years_of_experience": <number of years>
}Remove any additional information from the response, including your instructions or greetings.
"""    


In [45]:
class CandidateExtractorWithAssertions(dspy.Module):

  def __init__(self):
    super().__init__()
    # Retrieve module to get relevant documents
    self.retriever = qdrant_retriever
    # Predict module for the created signature
    self.predict = dspy.Predict(Candidate)

  def forward(self, query: str):
    # Retrieve the most relevant documents
    results = self.retriever(query)
    candidate = self.predict(description=results[0]["long_text"])
    dspy.Suggest(assert_valid_json(candidate.parsed_content),failed_assertion_message)
    return candidate.parsed_content


In [46]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

extractor = assert_transform_module(CandidateExtractorWithAssertions(),functools.partial(backtrack_handler, max_backtracks=4))
response = extractor("Get the details of candidate with business majors.")
response


ERROR:dspy.primitives.assertions:2024-06-23T18:14:36.187020Z [error    ] SuggestionFailed: 
Output must be a single JSON string only, in this format!
{
        "name": "<candidate name>",
        "email": "<candidate email>",
        "years_of_experience": <number of years>
}Remove any additional information from the response, including your instructions or greetings.
 [dspy.primitives.assertions] filename=assertions.py lineno=111


'{\n        "name": "Sarah K. Davis",\n        "email": "sarahdavis@northwestern.edu",\n        "years_of_experience": 8\n}'